<a href="https://colab.research.google.com/github/tho071206-ther/TriTueNhanTao/blob/main/TOMAU1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import numpy as np

# --- 1. CHƯƠNG TRÌNH CON: HÀM ĐỌC FILE (Bài 1) ---
def read_graph_from_file(file_path):
    """Đọc ma trận kề từ file .txt và tạo tên đỉnh."""
    G = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Xử lý các giá trị số nguyên trong file
                row = [int(val) for val in line.strip().split() if val.strip()]
                if row:
                    G.append(row)

        # Kiểm tra tính hợp lệ và tạo tên đỉnh A, B, C...
        if not G or len(G) != len(G[0]):
            raise ValueError("Ma trận kề không hợp lệ (Không phải ma trận vuông).")

        num_nodes = len(G)
        node_names = string.ascii_uppercase[:num_nodes]

        # Tạo ánh xạ tên đỉnh sang chỉ số (dùng cho việc tra cứu ma trận kề)
        node_to_index = {name: i for i, name in enumerate(node_names)}

        return G, node_names, node_to_index

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file tại đường dẫn {file_path}")
        return None, None, None
    except ValueError as e:
        print(f"Lỗi khi đọc ma trận: {e}")
        return None, None, None

# --- 2. CHƯƠNG TRÌNH CON: TÍNH BẬC VÀ SẮP XẾP (Bài 2) ---
def calculate_degree(G):
    """Tính bậc (degree) của mỗi đỉnh."""
    # Bậc của một đỉnh là tổng các cạnh kề (tổng hàng) [cite: 88]
    degree = [sum(row) for row in G]
    return degree

def sort_nodes_by_degree(node_names, degree):
    """Sắp xếp các đỉnh theo bậc giảm dần (Heuristic Bậc lớn nhất)."""
    # Ghép bậc và chỉ số [(bậc, chỉ_số)]
    degree_index_pairs = [(degree[i], i) for i in range(len(degree))]

    # Sắp xếp giảm dần theo bậc
    degree_index_pairs.sort(key=lambda item: item[0], reverse=True)

    # Trích xuất thứ tự tên đỉnh
    sorted_nodes = [node_names[index] for (deg, index) in degree_index_pairs]
    return sorted_nodes

# --- 3. CHƯƠNG TRÌNH CON: THUẬT TOÁN TÔ MÀU (Bài 2) ---
def greedy_coloring(G, node_names, sorted_nodes, node_to_index):
    """Thực hiện thuật toán tô màu tham lam với Forward Checking."""
    AVAILABLE_COLORS = ["Blue", "Red", "Yellow", "Green", "Purple", "Orange", "Cyan", "Magenta"]

    # Khởi tạo miền giá trị (Domain) cho mỗi đỉnh [cite: 91, 93]
    color_domain = {name: list(AVAILABLE_COLORS) for name in node_names}

    # Lưu trữ kết quả tô màu
    the_solution = {}

    for current_node in sorted_nodes:
        # Chọn màu (màu đầu tiên khả dụng trong miền giá trị) [cite: 112]
        if not color_domain[current_node]:
            print(f"LỖI: Đỉnh {current_node} không còn màu khả dụng! Không thể tô màu.")
            return None

        chosen_color = color_domain[current_node][0]
        the_solution[current_node] = chosen_color

        # Forward Checking: Loại bỏ màu đã chọn khỏi các đỉnh kề [cite: 115, 116]
        current_index = node_to_index[current_node]

        for j in range(len(G)):
            # Kiểm tra nếu j là đỉnh kề (G[current_index][j] == 1)
            if G[current_index][j] == 1:
                neighbor_node = node_names[j]

                # Chỉ xử lý các đỉnh CHƯA tô màu
                if neighbor_node not in the_solution:
                    if chosen_color in color_domain[neighbor_node]:
                        color_domain[neighbor_node].remove(chosen_color)

    return the_solution

# --- CHƯƠNG TRÌNH CHÍNH (MAIN) CHO BÀI 1 & 2 ---
if __name__ == "__main__":
    FILE_PATH = "input_graph.txt"

    # Tạo file mẫu để test (theo ma trận kề trong tài liệu) [cite: 44, 47-78]
    # Lưu ý: Trong môi trường thực tế, sinh viên cần tạo file này.
    try:
        with open(FILE_PATH, 'w') as f:
            f.write("0 1 1 0 1 0\n")
            f.write("1 0 1 1 0 1\n")
            f.write("1 1 0 1 1 0\n")
            f.write("0 1 1 0 0 1\n")
            f.write("1 0 1 0 0 1\n")
            f.write("0 1 0 1 1 0\n")
        print(f"Đã tạo file mẫu {FILE_PATH} để kiểm tra.")
    except IOError:
        print("Lỗi: Không thể tạo file mẫu.")

    print("\n--- BÀI TẬP TẠI LỚP (BÀI 1 & 2) ---")

    G, node_names, node_to_index = read_graph_from_file(FILE_PATH)

    if G and node_names:
        print(f"Đồ thị đọc được có {len(G)} đỉnh: {node_names}")

        degrees = calculate_degree(G)
        sorted_nodes = sort_nodes_by_degree(node_names, degrees)
        print(f"Bậc các đỉnh: {degrees}")
        print(f"Thứ tự tô màu ưu tiên (Bậc giảm dần): {sorted_nodes}")

        result_solution = greedy_coloring(G, node_names, sorted_nodes, node_to_index)

        print("-" * 50)
        if result_solution:
            print("✅ KẾT QUẢ TÔ MÀU ĐỒ THỊ:")
            # In kết quả theo thứ tự bảng chữ cái để dễ nhìn [cite: 118]
            for node, color in sorted(result_solution.items()):
                print(f"Đỉnh {node} = {color}") [cite: 119]

            colors_used = set(result_solution.values())
            print(f"\nTổng số màu đã dùng: {len(colors_used)}")
        else:
            print("❌ KHÔNG TÌM ĐƯỢC LỜI GIẢI thỏa mãn ràng buộc.")